In [1]:
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index.llms import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from huggingface_hub import login
import torch
import os

os.chdir("D:\Projects\RAG-webapp")

In [2]:
def load_model(model_name="TheBloke/Llama-2-7b-Chat-GPTQ", device='gpu'):
    # setting device
    if device == 'gpu':
        gpu=0
        device = torch.device(f"cuda:{gpu}" if torch.cuda.is_available() else "cpu")
        if torch.cuda.is_available():
            torch.cuda.set_device(device)
        torch.cuda.get_device_name(0)
    elif device == 'cpu':
        device = torch.device('cpu')
        torch.cuda.set_device(device)

    with open('huggingface_credentials.txt', 'r') as file:
        hf_token = file.readline().strip()

    login(token=hf_token)

    # Create tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name
        ,device_map='cuda'                 
        )

    # Define model
    model = AutoModelForCausalLM.from_pretrained(model_name
        # ,cache_dir=r"C:\Users\henry\.cache\huggingface\hub"
        # ,cache_dir=r"C:\Users\user2\.cache\huggingface\hub"
        ,device_map='cuda'  
        # , torch_dtype=torch.float16
        # ,low_cpu_mem_usage=True
        # ,rope_scaling={"type": "dynamic", "factor": 2}
        # ,load_in_8bit=True,
        ).to(device)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    model_obj = {"model": model, "tokenizer": tokenizer, "streamer": streamer, "device": device,  }

    return model_obj


In [3]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"

model_obj = load_model(model_name)
model = model_obj["model"]
tokenizer = model_obj["tokenizer"]
device = model_obj["device"]
streamer = model_obj["streamer"]



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


D:\Projects\venv2\lib\site-packages\awq\modules\linear\exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: DLL load failed while importing exl_ext: The specified module could not be found.
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
D:\Projects\venv2\lib\site-packages\awq\modules\linear\exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: DLL load failed while importing exlv2_ext: The specified module could not be found.
  warnings.warn(f"AutoAWQ could not load ExLlamaV2 kernels extension. Details: {ex}")
D:\Projects\venv2\lib\site-packages\awq\modules\linear\gemm.py:14: UserWarning: AutoAWQ could not load GEMM kernels extension. Details: DLL load failed while importing awq_ext: The specified module could not be found.
  warnings.warn(f"AutoAWQ could not load GEMM kernels extension. Details: {ex}")
D:\Projects\venv2\lib\site-packages\awq\modules\linear\gemv.py:11: UserWarnin

In [4]:
system_prompt = """<s>[INST] <<SYS>>
        You are a helpful, respectful and honest assistant. Always answer as
        helpfully as possible, while being safe.`
        If a question does not make any sense, or is not factually coherent, explain
        why instead of answering something not correct. If you don't know the answer
        to a question, please don't share false information.
        Try to be exact in information and numbers you tell.
        Your goal is to provide answers completely based on the information provided
        and if you use yourown knowledge please inform the user.
        and it is important to respond as breifly as possible.<</SYS>>
        # """

query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    model=model,
    tokenizer=tokenizer
)

In [ ]:
# Get the response object (with streaming enabled in your query engine)
response = self.query_engine.query(msg)

# The response object has a .response_gen generator for streaming tokens/chunks
try:
    response_gen = response.response_gen
except AttributeError:
    # Fallback if streaming is not enabled or not supported
    yield "%%%END%%%"
    return

# Stream the response token by token/chunk by chunk
try:
    while True:
        chunk = next(response_gen)
        # Do something with chunk, e.g., send to websocket
        print(chunk, end="", flush=True)
        # In async context, you might want to: await asyncio.sleep(0)
except StopIteration:
    pass

2

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

In [3]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", device_map="cuda")

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

In [4]:
# Prepare the messages
user_query = "What could be the most influencial scientific advancement or discovery in the entire era of human being?"
messages = [{"role": "user", "content": user_query}]

# Tokenize using chat template
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)

# Initialize the streamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Generate with streaming
_ = model.generate(
    **inputs,
    max_new_tokens=100,
    streamer=streamer,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
D:\Projects\venv2\lib\site-packages\transformers\models\mistral\modeling_mistral.py:473: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Identifying and understanding the theory of evolution, particularly the work of Charles Darwin, is often considered one of the most influential scientific advancements in human history. The theory of evolution through natural selection has fundamentally changed our understanding of the natural world and our place in it. It has shaped many fields of science, from biology and genetics to anthropology and archaeology. Additionally, it has had a significant impact on culture, philosophy, and religion.

However, it's important


Query keyword extractor

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ",
    device_map="cuda",
    torch_dtype=torch.float16  # Assuming you're using a GPU with FP16
)

# ----- Define system prompt -----
system_prompt = """You are a smart AI assistant helping with document retrieval.

Your job is to extract only the essential keywords and phrases from a user query that can be used to retrieve relevant documents. Do NOT rewrite the query. Instead, return a minimal list of distinct, meaningful terms or short phrases.

Do not include stopwords, pronouns, greetings, or irrelevant words. Focus on entities, topics, key terms, technical concepts, and specific content.

Return keywords as a comma-separated list, and nothing else.

Examples:

User: What are the effects of climate change on polar bear populations?
Keywords: climate change, polar bears, effects

User: How does the Transformer architecture work in deep learning?
Keywords: Transformer, deep learning, architecture

User: سلام، بگو ببینم حافظه کاری در روانشناسی چی هست؟
Keywords: حافظه کاری, روانشناسی

Now extract keywords from the following query:
"""

# ----- User query -----
user_query = "What are the main use cases of blockchain in the finance sector?"

# ----- Final prompt -----
prompt = f"{system_prompt}User: {user_query}\nKeywords:"

# ----- Tokenize -----
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# ----- Generate -----
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=30,
        temperature=0.7,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

# ----- Decode and print result -----
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("---- Raw Output ----")
print(generated_text)

# ----- Extract only the keyword list -----
# This will remove the prompt and just keep the generated keywords
extracted_keywords = generated_text.split("Keywords:")[-1].strip()
print("\n---- Extracted Keywords ----")
print(extracted_keywords)


CUDA extension not installed.
CUDA extension not installed.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


D:\Projects\venv2\lib\site-packages\transformers\modeling_utils.py:4664: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers

---- Raw Output ----
You are a smart AI assistant helping with document retrieval.

Your job is to extract only the essential keywords and phrases from a user query that can be used to retrieve relevant documents. Do NOT rewrite the query. Instead, return a minimal list of distinct, meaningful terms or short phrases.

Do not include stopwords, pronouns, greetings, or irrelevant words. Focus on entities, topics, key terms, technical concepts, and specific content.

Return keywords as a comma-separated list, and nothing else.

Examples:

User: What are the effects of climate change on polar bear populations?
Keywords: climate change, polar bears, effects

User: How does the Transformer architecture work in deep learning?
Keywords: Transformer, deep learning, architecture

User: سلام، بگو ببینم حافظه کاری در روانشناسی چی هست؟
Keywords: حافظه کاری, روانشناسی

Now extract keywords from the following query:
User: What are the main use cases of blockchain in the finance sector?
Keywords: bloc

In [2]:
extracted_keywords


'blockchain, finance sector, use cases'

In [ ]:
k